In [1]:
%load_ext autoreload
%autoreload 2
from confignb import *

X, y, pts, ks, masks = util.load_data('data/processed/ASCADv.h5')

In [ ]:
pl = make_pipeline(
    SelectFromModel(simplerf(SEED, depth=10, n_estimators=100), max_features=200),
    StandardScaler(),
    util.InteractionsSelectKBest(k=2000)
)

splits_inter_all = util.precompute_splits(
    SEED, pl, X, y, pts, ks, masks
)
pickle.dump(splits_inter_all, open('data/processed/splits_inter_all.pkl', 'wb'))

  0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
pl_lr_poly = []

for k in [300, 500, 700, 1000]:
    pl = make_pipeline(
        SelectKBest(f_classif, k=k),
        StandardScaler(),
        LogisticRegression(C=0.01, tol=1e-3, random_state=SEED, verbose=1)
    )
    results = cv_precomputed(pl, splits_inter_all, verbose=False)
    pl_lr_poly.append(util.results_to_dict(results, k=k))

    logging.info(f"[{k}]: {results['atk_score_mean'].mean():.4f}")
    pd.DataFrame(pl_lr_poly).to_csv('data/lr_poly.csv', index=False)

In [ ]:
res = cv_precomputed(simplerf(SEED, 10), splits_inter_all, verbose=True)